# Orekit features

A certain number of features in `ephemerista` are based on the [Orekit library](https://www.orekit.org/), and in particular on its [Jpype Python wrapper](https://pypi.org/project/orekit-jpype/) which was recently made available on PyPi.
Though available in Python, the Orekit Python wrapper interacts with a Java virtual machine in the background, therefore `ephemerista` also depends on a JDK, which is usually provided by [jdk4py](https://pypi.org/project/jdk4py/), available as a dependency of `ephemerista`.

First, a CSV file containing Earth Orientation Parameters (EOP) must be passed to `ephemerista`:

In [ ]:
import ephemerista

ephemerista.init(eop_path="../tests/resources/finals2000A.all.csv", spk_path="../tests/resources/de440s.bsp")

## Propagation

`ephemerista`'s `NumericalPropagator` and `SemiAnalyticalPropagator` inherit from the same base class, `OrekitPropagator`, because most of the code is similar between both. They are based on their Orekit counterparts, [NumericalPropagator](https://www.orekit.org/static/apidocs/org/orekit/propagation/numerical/NumericalPropagator.html) and [DSSTPropagator](https://www.orekit.org/static/apidocs/org/orekit/propagation/semianalytical/dsst/DSSTPropagator.html):

* The `NumericalPropagator` is a full-fledged numerical propagator based on a [Dormand-Prince 8(5,3) variable step integrator](https://www.hipparchus.org/apidocs/org/hipparchus/ode/nonstiff/DormandPrince853Integrator.html), featuring most perturbation force models:
  * [Holmes-Featherstone gravitational model](https://www.orekit.org/static/apidocs/org/orekit/forces/gravity/HolmesFeatherstoneAttractionModel.html), up to an arbitrary degree and order.
  * [Third-body attraction](https://www.orekit.org/static/apidocs/org/orekit/forces/gravity/ThirdBodyAttraction.html) from any planet of the solar system, the Sun or the Moon
  * [Solar radiation pressure](https://www.orekit.org/static/apidocs/org/orekit/forces/radiation/SolarRadiationPressure.html), for now isotropic
  * Atmospheric drag based on the [NRLMSISE00 density model](https://www.orekit.org/static/apidocs/org/orekit/models/earth/atmosphere/NRLMSISE00.html), using three-hourly [CSSI space weather data](https://www.orekit.org/static/apidocs/org/orekit/models/earth/atmosphere/data/CssiSpaceWeatherData.html). For now isotropic.

* The `SemiAnalyticalPropagator` uses the Orekit implementation of the Draper Semi-analytical Satellite Theory (DSST), which describes a semianalytical propagator that combines the accuracy of numerical propagators with the speed of analytical propagators. It features similar force models to the `NumericalPropagator`, and the Python parameters are the same.

### Initialization

The propagator's initial state vector must be provided as a `TwoBody` object, either in `Keplerian` or in `Cartesian` representation.

#### From `Cartesian` (position, velocity)

In [ ]:
import numpy as np

from ephemerista.coords.twobody import Cartesian
from ephemerista.time import Time

time_start = Time.from_iso("TDB", "2016-05-30T12:00:00")

r = np.array([6068.27927, -1692.84394, -2516.61918])  # km
v = np.array([-0.660415582, 5.495938726, -5.303093233])  # km/s

state_init = Cartesian.from_rv(time_start, r, v)
display(state_init)

#### From `Keplerian`

In [ ]:
from ephemerista.angles import Angle
from ephemerista.coords.anomalies import TrueAnomaly
from ephemerista.coords.shapes import RadiiShape
from ephemerista.coords.twobody import Inclination, Keplerian
from ephemerista.time import Time

time_start = Time.from_iso("TDB", "2016-05-30T12:00:00")

state_init = Keplerian(
    time=time_start,
    shape=RadiiShape(ra=10000.0, rp=6800.0),  # km
    inc=Inclination(degrees=98.0),
    node=Angle(degrees=0.0),
    arg=Angle(degrees=0.0),
    anomaly=TrueAnomaly(degrees=90.0),
)
display(state_init)

### Declaring and configuring a propagator

The propagator takes at least the initial state as parameter. The initial state can later be overriden though by calling the method `set_initial_state`.

Declaring a simple propagator with default parameters, in either the numerical or the semi-analytical flavour:

In [ ]:
from ephemerista.propagators.orekit.numerical import NumericalPropagator

propagator = NumericalPropagator(state_init=state_init)

In [ ]:
from ephemerista.propagators.orekit.semianalytical import SemiAnalyticalPropagator

propagator = SemiAnalyticalPropagator(state_init=state_init)

The constructor internally starts the JVM for Orekit java, set ups some physical data (e.g. JPL DE440 kernels, etc.), and configures the force models.

### General propagator configuration

The following `float` arguments can be passed to the `NumericalPropagator` or `SemiAnalyticalPropagator` constructors for general configuration of the propagators:

* `prop_min_step`: Minimum integrator step, default is 1 millisecond. [Reference](https://www.hipparchus.org/apidocs/org/hipparchus/ode/nonstiff/DormandPrince853Integrator.html#%3Cinit%3E(double,double,double%5B%5D,double%5B%5D))
* `prop_max_step`: Maximum integrator step, default is 3600 seconds. [Reference](https://www.hipparchus.org/apidocs/org/hipparchus/ode/nonstiff/DormandPrince853Integrator.html#%3Cinit%3E(double,double,double%5B%5D,double%5B%5D))
* `prop_init_step` Initial integrator step, default is 60 seconds. [Reference](https://www.hipparchus.org/apidocs/org/hipparchus/ode/nonstiff/DormandPrince853Integrator.html#%3Cinit%3E(double,double,double%5B%5D,double%5B%5D))
* `prop_position_error`: Order of magnitude of the desired position error of the integrator for a sample LEO orbit, default is 10 meters. Decreasing this value can improve the propagation accuracy for complex force models, but will slow down the propagator. [Reference](https://www.orekit.org/static/apidocs/org/orekit/propagation/numerical/NumericalPropagator.html#tolerances(double,org.orekit.orbits.Orbit,org.orekit.orbits.OrbitType))

* `mass`: spacecraft mass in kilograms, default is 1000 kilograms.

### Force model configuration

Without any arguments passed to the constructor, the default force model uses Earth spherical harmonics up to degree and order 4. As both the `NumericalPropagator` and `SemiAnalyticalPropagator` use the same arguments, in the following we will show only examples with the `NumericalPropagator`.

#### Loading a custom gravity model

Only the [4 file formats supported by Orekit](https://www.orekit.org/static/apidocs/org/orekit/forces/gravity/potential/PotentialCoefficientsReader.html) are available in `ephemerista`. For instance to load a ICGEM file:

In [ ]:
from pathlib import Path

propagator = NumericalPropagator(state_init=state_init, gravity_file=Path("ICGEM_GOCO06s.gfc"))

For configuring the degree and order of the gravity model, see the next section below.

#### Gravitational model

The following example configures gravity spherical harmonics with degree and order 64.

In [ ]:
propagator = NumericalPropagator(state_init=state_init, grav_degree_order=(64, 64))

The following disables the spherical harmonics to only keep a Keplerian model.

In [ ]:
propagator = NumericalPropagator(state_init=state_init, grav_degree_order=None)

#### Third-body perturbations

The following adds the Sun as a third-body perturbation.

In [ ]:
from ephemerista.bodies import Origin

propagator = NumericalPropagator(state_init=state_init, third_bodies=[Origin(name="Sun")])

The following adds the Sun, the Moon and Jupiter as third-body perturbators.

In [ ]:
propagator = NumericalPropagator(
    state_init=state_init, third_bodies=[Origin(name="Sun"), Origin(name="luna"), Origin(name="jupiter")]
)

#### Solar radiation pressure

Simply use the `enable_srp` flag, which is `False` by default.

Use `cross_section` to set the spacecraft cross-section in m^2 and `c_r` to set the reflection coefficient. [Reference](https://www.orekit.org/static/apidocs/org/orekit/forces/radiation/IsotropicRadiationSingleCoefficient.html)

In [ ]:
propagator = NumericalPropagator(state_init=state_init, enable_srp=True, cross_section=0.42, c_r=0.8)

#### Atmospheric drag

Simply use the `enable_drag` flag, which is `False` by default.

Use `cross_section` to set the spacecraft cross-section in m^2 and `c_d` to set the drag coefficient. [Reference](https://www.orekit.org/static/apidocs/org/orekit/forces/drag/IsotropicDrag.html)

In [ ]:
propagator = NumericalPropagator(state_init=state_init, enable_drag=True, cross_section=0.42, c_d=2.1)

#### Full-fledged force model

In [ ]:
propagator = NumericalPropagator(
    state_init=state_init,
    mass=430.5,
    cross_section=0.42,
    c_d=2.1,
    c_r=0.8,
    grav_degree_order=(64, 64),
    third_bodies=[Origin(name="Sun"), Origin(name="luna"), Origin(name="jupiter")],
    enable_srp=True,
    enable_drag=True,
)

## Propagator usage

### Propagating to a single date

To perform a single propagation to a date:

In [ ]:
time_end = Time.from_iso("TDB", "2016-05-30T12:01:00")

state_end = propagator.propagate(time=time_end)
display(state_end)

### Propagating from a list of `Time`

To retrieve all the intermediary state vectors between the initial and the final states, pass a list of `Time` objects to the propagator's `propagate` method via the `time` argument:

In [ ]:
time_end = Time.from_iso("TDB", "2016-05-30T16:00:00")
t_step = 60.0  # s
time_list = time_start.trange(time_end, t_step)

trajectory = propagator.propagate(time=time_list)
display(trajectory)

## 3D trajectory visualization

This feature is actually available for other propagators such as `SGP4`.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(trajectory.origin.plot_3d_surface())
fig.add_trace(trajectory.plot_3d())

fig.update_layout(
    title=f"3D trajectory, {trajectory.origin.name}-centered {trajectory.frame.abbreviation} frame",
)
fig.show()

For visualizing ground tracks, take a look at the `groundtracks.ipynb` notebook.

## Stopping conditions

It is possible to stop propagation at events defined in the enum `ephemerista.propagators.events.StoppingEvent`. As of now, apoapsis and periapsis can be used as stopping conditions. The example below shows how to stop the propagation at periapsis. The same can be done for the apoapsis using the enum value `StoppingEvent.APOAPSIS` instead of `StoppingEvent.PERIAPSIS`.

In [ ]:
from ephemerista.angles import Angle
from ephemerista.coords.anomalies import TrueAnomaly
from ephemerista.coords.shapes import RadiiShape
from ephemerista.coords.twobody import Inclination, Keplerian
from ephemerista.propagators.events import StoppingEvent
from ephemerista.time import Time

time_start = Time.from_iso("TDB", "2016-05-30T12:00:00")

state_init = Keplerian(
    time=time_start,
    shape=RadiiShape(ra=10000.0, rp=6800.0),  # km
    inc=Inclination(degrees=98.0),
    node=Angle(degrees=0.0),
    arg=Angle(degrees=0.0),
    anomaly=TrueAnomaly(degrees=90.0),
)

time_end = Time.from_iso("TDB", "2016-05-30T14:00:00")
t_step = 60.0  # s
time_list = time_start.trange(time_end, t_step)

propagator.set_initial_state(state_init)
trajectory = propagator.propagate(time=time_list, stop_conds=[StoppingEvent.PERIAPSIS])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(trajectory.origin.plot_3d_surface())
fig.add_trace(trajectory.plot_3d())

fig.update_layout(
    title=f"3D trajectory, {trajectory.origin.name}-centered {trajectory.frame.abbreviation} frame",
)
fig.show()

## CCSDS OEM/OPM/OMM import/export

`ephemerista` is able to read and write CCSDS OEM, OPM and OMM messages among others.

### Writing CCSDS OEM files

`ephemerista` supports writing CCSDS files in both KVN and XML formats, relying on Orekit in the background.

In [ ]:
from ephemerista.propagators.orekit.ccsds import CcsdsFileFormat, write_oem

dt = 300.0
write_oem(trajectory, "oem_example_out.txt", dt, CcsdsFileFormat.KVN)
write_oem(trajectory, "oem_example_out.xml", dt, CcsdsFileFormat.XML)

with open("oem_example_out.txt") as f:
    display(f.readlines()[0:25])

### Reading CCSDS OEM files

In [ ]:
from ephemerista.propagators.orekit.ccsds import parse_oem

dt = 300.0
trajectory = parse_oem("OEMExample5.txt", dt)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(trajectory.origin.plot_3d_surface())
fig.add_trace(trajectory.plot_3d())

fig.update_layout(
    title=f"3D trajectory, {trajectory.origin.name}-centered {trajectory.frame.abbreviation} frame",
)
fig.show()